## load the CSV file

In [ ]:
import pandas as pd

# Load the Bitcoin data
data = pd.read_csv('data/BTC_data.csv')

# Display the first few rows
data.head()


## Preprocessing the Data     

The next step in the notebook will be preprocessing the data. Preprocessing might include handling missing values, normalizing data, and removing unwanted columns.       

In [ ]:
# Drop any missing values
data = data.dropna()

# Normalize the 'Close' prices to a range of 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data['Close'] = scaler.fit_transform(data[['Close']])

# Display the cleaned data
data.head()

### Explanation:         

- *dropna():* Removes rows with missing values.         
- *MinMaxScaler:* This function from sklearn scales the data to be between 0 and 1, which helps the neural network during training.    

## Setting Up the tf.data.Dataset for Model Inputs     

Next, we’ll use TensorFlow’s tf.data.Dataset to prepare the data for the model. This is how we can structure our time series data.

In [ ]:
import tensorflow as tf

# Create dataset for time series prediction
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data)-time_step-1):
        X.append(data[i:(i+time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Convert the dataframe to a numpy array
dataset = data['Close'].values.reshape(-1,1)

# Set time step to 60 (meaning we'll look at 60 previous days to predict the next one)
time_step = 60
X, y = create_dataset(dataset, time_step)

### Explanation:    

We’re splitting the data into inputs (X) and outputs (y), where X is a sequence of past values and y is the next value we want to predict.  

## Model Architecture       

We’ll use an LSTM (Long Short-Term Memory) model, which is great for handling time series data.

In [ ]:
# Build LSTM Model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    tf.keras.layers.LSTM(50, return_sequences=False),
    tf.keras.layers.Dense(25),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model summary
model.summary()

### Explanation:   

- *LSTM:* A type of RNN that helps in capturing patterns in time series data.     
- *Dense:* A fully connected layer that outputs the predicted value.


## Results and Evaluation    
A
fter training the model, let's evaluate its performance and visualize the results.

In [ ]:
# Make predictions
predictions = model.predict(X_test)

# Inverse scaling to get actual price predictions
predictions = scaler.inverse_transform(predictions)

# Plot the results
plt.plot(data['Close'].values)
plt.plot(predictions)
plt.show()